# RangRang Notebook

<p align="center">
  <img src="https://raw.githubusercontent.com/Hyuto/rangrang-ML/master/assets/logo.png" alt="logo" width="300px" height="300px" />
</p>

This is the notebook that Machine Learning team of `B21-CAP0378` use to develop ML side of the App

## Content

1. Training Object Detection Model Using Tensorflow Object Detection API
2. Converting Saved Model to `tflite`

## Environtment

Running on google colab with `gpu` enabled

**Setup** `tensorflow object detection API`

In [ ]:
!mkdir TensorFlow 
%cd TensorFlow
!git clone https://github.com/tensorflow/models.git
%cd models/research
!protoc object_detection/protos/*.proto --python_out=.
!pip install cython
!pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .
!python object_detection/builders/model_builder_tf2_test.py
%cd ../../..

**Clone our repository for main directory**

In [ ]:
!git clone https://github.com/Hyuto/rangrang-ML.git
!cp -r rangrang-ML/workspace TensorFlow
!cp -r rangrang-ML/pre-trained-models rangrang-ML/scripts TensorFlow/workspace/color_detection
!cp -r rangrang-ML/pre-trained-models rangrang-ML/scripts TensorFlow/workspace/object_detection


## 1. Training Object Detection Model Using Tensorflow Object Detection API

Training object detection model. Model to be trained :

1. Color Detection
2. Object Detection

### 1.1 Training Color Detection Model

Training and fine tuned `ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8` for color detection model

**Setup main directory**

In [ ]:
%cd TensorFlow/workspace/color_detection
!mkdir exported-models

**Generate `tfrecord` from the dataset**

In [ ]:
# Generate tfrecord
!python scripts/preprocessing/generate_tfrecord.py -x images/train -l annotations/label_map.pbtxt -o annotations/train.record
!python scripts/preprocessing/generate_tfrecord.py -x images/test -l annotations/label_map.pbtxt -o annotations/test.record

**Train the model**

In [ ]:
!python model_main_tf2.py --model_dir=models/model_aku --pipeline_config_path=models/model_aku/pipeline.config

**Go back to the workspace directory**

In [ ]:
%cd ../../..

### 1.2 Training Object Detection Model

Training and fine tuned `ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8` for object detection model

**Setup main directory**

In [ ]:
%cd TensorFlow/workspace/object_detection
!mkdir exported-models

**Generate tfrecord from the dataset**

In [ ]:
# Generate tfrecord
!python scripts/preprocessing/generate_tfrecord.py -x images/train -l annotations/label_map.pbtxt -o annotations/train.record
!python scripts/preprocessing/generate_tfrecord.py -x images/test -l annotations/label_map.pbtxt -o annotations/test.record

**Train the model**

In [ ]:
!python model_main_tf2.py --model_dir=models/model_aku --pipeline_config_path=models/model_aku/pipeline.config

**Go back to the workspace directory**

In [ ]:
%cd ../../..

## 2. Converting Saved Model to `tflite`

Convert fine tuned model to `tflite` so the model is deployable to android

**Convert checkpoint based model to `saved model`**

In [ ]:
!python TensorFlow/models/research/color_detection/export_tflite_graph_tf2.py \
        --pipeline_config_path TensorFlow/workspace/color_detection/models/model_aku/pipeline.config \
        --trained_checkpoint_dir TensorFlow/workspace/color_detection/models/model_aku/ \
        --output_directory TensorFlow/workspace/color_detection/exported-models

**Convert `saved model` to tflite**

In [ ]:
import tensorflow as tf
import numpy as np

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model('TensorFlow/workspace/color_detection/exported-models/saved_model') 
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
tflite_model = converter.convert()

# Save the model.
with open('detect.tflite', 'wb') as f:
  f.write(tflite_model)

!du -sh detect.tflite

**Write `labelmap`**

In [ ]:
LABELS = [
    'merah',
    'biru',
    'hijau',
    'putih',
    'kuning',
    'orange',
    'pink',
    'hitam',
    'coklat',
    'ungu'
]

with open('labelmap.txt', 'w') as w:
  w.write('\n'.join(LABELS))

**Write metadata for the `tflite` model**

In [ ]:
!pip install tflite-support

In [ ]:
from tflite_support.metadata_writers import object_detector
from tflite_support.metadata_writers import writer_utils
from tflite_support import metadata

ObjectDetectorWriter = object_detector.MetadataWriter
MODEL_PATH = "detect.tflite"
LABEL_FILE = "labelmap.txt"
EXPORT_PATH = "detect.tflite"

writer = ObjectDetectorWriter.create_for_inference(
    writer_utils.load_file(MODEL_PATH), [127.5], [127.5], [LABEL_FILE])
writer_utils.save_file(writer.populate(), EXPORT_PATH)

# Verify the populated metadata and associated files.
displayer = metadata.MetadataDisplayer.with_model_file(EXPORT_PATH)
print("Metadata populated:")
print(displayer.get_metadata_json())
print("Associated file(s) populated:")
print(displayer.get_packed_associated_file_list())

**Download `tflite` model**

In [ ]:
from google.colab import files

files.download('detect.tflite')
files.download('labelmap.txt')